Rosa Garza, July 5, 2018 (Day 4)


In [382]:
import numpy as np
import matplotlib as mpl
import pandas as pd
import pylab as plt
import csv

In [384]:
gene_dat=pd.read_csv('vcf_FA.csv',index_col='Unnamed: 0',encoding='utf-8')
matrix=gene_dat.as_matrix()
#print(matrix.shape[0])
#matrix[0]

In [385]:
gene_mut=np.zeros((matrix.shape[0],int(np.ceil((matrix.shape[1])/2))))
#print(gene_mut.shape)

In [386]:
# will go through each row (# of observations)
for rows in range(matrix.shape[0]):
    for cols in range(0,matrix.shape[1]-1,2):
        # within those observations, it will go until the 2nd to last column (excludes phenotype)
        #sanity check (next 2 lines)
        #print('cols val is:',cols)
        #print('matrix values are: ',matrix[rows][cols],matrix[rows][cols+1])
        #checks to see if either the L or R gene has a mutation
        if(matrix[rows][cols]==1 or matrix[rows][cols+1]==1):
            #if it does, then in the copied matrix, a 1 will be input
            gene_mut[rows][int(cols/2)]=1
            #reason for int(cols/2) is to match up indexes with the pairing # that it's on
            #ex: 0&1 gene output val. will be in the 0 index of copied matrix
            # 2&3 gene output val. will be in the 1 index, 4*6 in the 2nd index
        #sanity check of what is being output
        #print('copy matrix val is:',gene_mut[rows][int(cols/2)], 'for row:',rows,'for col:',int(cols/2))

In [387]:
gene_mut[:,-1]=matrix[:,-1] # gets last column from original matrix, puts it as last column for copied matrix

In [388]:
gene_mut

array([[   1.,    0.,    1., ...,    1.,    0.,  196.],
       [   1.,    1.,    1., ...,    0.,    0.,  173.],
       [   1.,    0.,    1., ...,    0.,    1.,  198.],
       ..., 
       [   1.,    0.,    1., ...,    0.,    0.,  168.],
       [   1.,    0.,    1., ...,    0.,    0.,  155.],
       [   1.,    0.,    1., ...,    0.,    1.,  145.]])

In order to get help on opening and writing to a new csv file, I used the following link:
https://stackoverflow.com/questions/30938840/python-csv-read-file-and-select-columns-and-write-to-new-csv-file

as well as the link below: 

http://www.pythonforbeginners.com/systems-programming/using-the-csv-module-in-python/

In [390]:
# getting column names of genes without the L or R
names=gene_dat.columns
new_colnames=[]
for n in range(0,len(names),2):
    new_colnames.append(names[n].split("-")[0])
#print(new_colnames)

For the code below, this helped me,
https://stackoverflow.com/questions/11106536/adding-row-column-headers-to-numpy-arrays

In [392]:
copied_data=pd.DataFrame(gene_mut,index=gene_dat.index,columns=new_colnames)

In [393]:
copied_data.to_csv("copiedGeneData.csv")

PermissionError: [Errno 13] Permission denied: 'copiedGeneData.csv'

In [394]:
rdm13=pd.read_csv('copiedGeneData.csv',index_col='Unnamed: 0',encoding='utf-8')
rdm13=rdm13.as_matrix()


In [396]:
def num_gene_mutations(raw_data):
    n_ones=[]
    # 0 to number of total rows
    for row in range(0,(raw_data.shape[0])):
        #calculating each person's number of gene mutations, excluding last column with phenotype levels
        n_ones.append(sum(rdm13[row,:-1]))
    return n_ones

In [398]:
sum_gene_mut=num_gene_mutations(rdm13)
#print(sum_gene_mut)
#for printing total number of people based on number of gene mutations
#plt.hist(sum_gene_mut,bins=13)
#plt.show()

In [399]:
# returning matrix of people with 0's that are less than floor(13/2)
def minority_0matrix(RDM,n_ones):
    col=RDM.shape[1]
    zero_mat=[]
    #basically 0- number of people
    for i in range (0,len(n_ones)):
        #if the number of 0's is less than or equal to floor(13/2), meaning more 0s than 1s
        if((RDM.shape[1]-1)-n_ones[i]<=np.floor((col-1)/2)):
            #we will add that row to a list
            zero_mat.append(RDM[i,])
    #getting the list and reshaping it to be of #ppl by #columns
    return np.array(zero_mat).reshape(len(zero_mat),col)

In [403]:
# getting an an array of the opposites. We were looking at the number of 1's within each row (num of mutations
#each person had), then we decided to have a matrix which calcualted num of 0's
nzeros=13-np.asarray(sum_gene_mut)
#print(nzeros)

In [409]:
minority_0matrix(rdm13,nzeros).shape
# returns dimension of matrix, meaning there's a matrix 21 x 14 in which these people have 7+ gene mutations
# we will be flipping these matrices in the future
#rdm13.shape[1]

(21, 14)

In [411]:
# matrix in which people have 5 gene mutations in total
rdm13[np.asarray(sum_gene_mut)==5]

array([[   1.,    0.,    1.,    1.,    1.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    1.,  198.],
       [   1.,    0.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,
           0.,    0.,    0.,    0.,  176.],
       [   1.,    0.,    0.,    0.,    1.,    1.,    0.,    1.,    0.,
           1.,    0.,    0.,    0.,  164.],
       [   1.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,    0.,
           0.,    0.,    0.,    0.,  149.],
       [   1.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,    0.,
           0.,    0.,    0.,    0.,  155.]])